In [20]:
# Library import
import xml.etree.ElementTree as ET
import spacy
import string
import numpy as np
import json
import random
import pickle

In [46]:
xml_filepath = './dataset/ABSA16_Restaurants_Train_SB1_v2.xml'
# create element tree object
tree = ET.parse(xml_filepath)

# get root element
root = tree.getroot()

In [47]:
# loading data from XML format to usable format -> review_text, entity, attribute, polarity(0/1)
data = []
max_review_length = 0
for it1 in root.findall('Review'):
    review = []
    for it2 in it1.findall('sentences'):
        for it3 in it2.findall('sentence'):
            for it4 in it3.findall('text'):
                review_text = it4.text.lower()

            for it4 in it3.findall('Opinions'):
                for it5 in it4.findall('Opinion'):
                    lst = [review_text]
                    txt = it5.attrib['category']
                    splitted_category = txt.split('#')
                    for tok in splitted_category:
                        lst.append(tok.lower())
                    
                    polarity = 0
                    if it5.attrib['polarity'] == 'positive':
                        polarity = 1
                    lst.append(polarity)
                    review.append(lst)

    max_review_length = max(max_review_length,len(review))
    if len(review) != 0:
        data.append(review)

print('Max review length is: ', max_review_length)

Max review length is:  44


In [48]:
print(len(data))

349


In [26]:
# sm is small here, can use large here as lg here by downloading the large file
# pip install -U pip setuptools wheel
# pip install -U spacy
# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [49]:
# removed punctuations
max_sent_length = 0
for i in range(len(data)):
    for j in range(len(data[i])):
        sent = data[i][j][0]

        token_lst = []
        for it in nlp(sent):
            token = it.text
            str = ''
            for ch in token:
                if ch == "'" or ch not in string.punctuation:
                    if ch == "'":
                        str += ' '
                    str += ch
                else:
                    str += ' '
            lst = str.split()
            for token in lst:
                token_lst.append(token)

        max_sent_length = max(len(token_lst), max_sent_length)
        data[i][j][0] = token_lst

print('Maximum Sentence Lenght is: ',max_sent_length)

Maximum Sentence Lenght is:  61


In [50]:
non_padded_sent_length = []
for i in range(len(data)):
    non_padded_sent_length.append(len(data[i]))

In [51]:
# review level padded thing
padded_sent = []
for i in range(max_sent_length):
    padded_sent.append('pad')
padded_review = [padded_sent,'pad','pad',0]

# sentence level and review level padding
padded_data = []
temp_data = data.copy()
for i in range(len(data)):
    for j in range(max_review_length):
        if j < len(data[i]):
            token_lst = data[i][j][0]
            sent = []
            for k in range(max_sent_length):
                if k < len(token_lst):
                    sent.append(token_lst[k])
                else:
                    sent.append('pad')
            temp_data[i][j][0] = sent
        
        else:
            temp_data[i].append(padded_review)

    # inserting a token to acknowledge that after this point, only padded sentences occur
    padded_data.append([temp_data[i],non_padded_sent_length[i]])

In [52]:
def check_and_get_embedding(token):
    if token not in emmbed_dict:
        vec = []
        for _ in range(emmbedding_dimensions):
            vec.append(random.uniform(-1,1))
        emmbed_dict[token] = np.array(vec)
        
    return emmbed_dict[token]

In [53]:
# 200 dimensions vectors for the tokens. Giving 0's to padding token and generating randomly for oov tokens.
emmbedding_dimensions = 200

emmbed_dict = {}
emmbed_dict['pad'] = np.zeros(emmbedding_dimensions)
with open('./dataset/200d_glove.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:],'float32')
        emmbed_dict[word]=vector


review_embedding = []
for i in range(len(padded_data)):
    reviews = []
    for j in range(len(padded_data[i][0])):
        review = []
        
        lst = []
        for token in padded_data[i][0][j][0]:
            if token not in emmbed_dict:
                vec = []
                for i in range(emmbedding_dimensions):
                    vec.append(random.uniform(-1,1))
                emmbed_dict[token] = np.array(vec)

            lst.append(emmbed_dict[token])

        # words embeddings
        review.append(np.array(lst))
        # aspect embeddings
        review.append(0.5*(check_and_get_embedding(data[i][j][1]) + check_and_get_embedding(data[i][j][2])))
        # polarity
        review.append(data[i][j][3])

        reviews.append(review)

    review_embedding.append([reviews,padded_data[i][1]])

In [54]:
# not converted opinions into embeddings -> not known how to handle
dump_this_dict = {}
for i in range(len(review_embedding)):
    dump_this_dict[i] = review_embedding[i]

In [57]:
with open('train_restaurants.json', 'wb') as f:
    pickle.dump(dump_this_dict, f)

In [58]:
with open('train_restaurants.json', 'rb') as f:
    data = pickle.load(f)